### PEAS - Data structures and fringes that define the Agent environment goes here

In [1]:
#Code block
# Define the Grid and Players
import numpy as np

# Constants for the game
ROWS = 6
COLS = 7
EMPTY = 0
PLAYER_HUMAN = 1
PLAYER_AI = 2

# Create a 7x6 grid (2D array)
def create_grid():
    return np.zeros((ROWS, COLS))

# Drop a disc in the chosen column
def drop_disc(grid, row, col, disc):
    grid[row][col] = disc

# Check if a column has space for a disc
def is_valid_location(grid, col):
    return grid[ROWS-1][col] == 0

# Get the next available row in the column
def get_next_open_row(grid, col):
    for r in range(ROWS):
        if grid[r][col] == 0:
            return r

# Print the grid (reversed to show correctly in terminal)
def print_grid(grid):
    print(np.flip(grid, 0))

# Check if the board is full
def is_board_full(grid):
    return all(grid[ROWS-1][c] != 0 for c in range(COLS))

# Example of initializing the grid
grid = create_grid()
print_grid(grid)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


### Implementation of the Min-Max algorithm

In [2]:
#Code block
import math

# Check for a win (horizontally, vertically, or diagonally)
def winning_move(grid, disc):
    # Check horizontal locations for win
    for c in range(COLS-3):
        for r in range(ROWS):
            if grid[r][c] == disc and grid[r][c+1] == disc and grid[r][c+2] == disc and grid[r][c+3] == disc:
                return True

    # Check vertical locations for win
    for c in range(COLS):
        for r in range(ROWS-3):
            if grid[r][c] == disc and grid[r+1][c] == disc and grid[r+2][c] == disc and grid[r+3][c] == disc:
                return True

    # Check positively sloped diagonals
    for c in range(COLS-3):
        for r in range(ROWS-3):
            if grid[r][c] == disc and grid[r+1][c+1] == disc and grid[r+2][c+2] == disc and grid[r+3][c+3] == disc:
                return True

    # Check negatively sloped diagonals
    for c in range(COLS-3):
        for r in range(3, ROWS):
            if grid[r][c] == disc and grid[r-1][c+1] == disc and grid[r-2][c+2] == disc and grid[r-3][c+3] == disc:
                return True
    return False

# Score the board based on the current game state
def evaluate_window(window, disc):
    score = 0
    opponent_disc = PLAYER_HUMAN if disc == PLAYER_AI else PLAYER_AI

    if window.count(disc) == 4:
        score += 100
    elif window.count(disc) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(disc) == 2 and window.count(EMPTY) == 2:
        score += 2

    if window.count(opponent_disc) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

# Static evaluation function for the grid
def score_position(grid, disc):
    score = 0

    # Center column preference
    center_array = [int(i) for i in list(grid[:, COLS//2])]
    center_count = center_array.count(disc)
    score += center_count * 3

    # Score Horizontal
    for r in range(ROWS):
        row_array = [int(i) for i in list(grid[r,:])]
        for c in range(COLS-3):
            window = row_array[c:c+4]
            score += evaluate_window(window, disc)

    # Score Vertical
    for c in range(COLS):
        col_array = [int(i) for i in list(grid[:,c])]
        for r in range(ROWS-3):
            window = col_array[r:r+4]
            score += evaluate_window(window, disc)

    # Score positive diagonal
    for r in range(ROWS-3):
        for c in range(COLS-3):
            window = [grid[r+i][c+i] for i in range(4)]
            score += evaluate_window(window, disc)

    # Score negative diagonal
    for r in range(ROWS-3):
        for c in range(COLS-3):
            window = [grid[r+3-i][c+i] for i in range(4)]
            score += evaluate_window(window, disc)

    return score

# Get all valid locations (columns with space)
def get_valid_locations(grid):
    valid_locations = []
    for col in range(COLS):
        if is_valid_location(grid, col):
            valid_locations.append(col)
    return valid_locations

# Minimax algorithm with fixed depth (3)
def minimax(grid, depth, alpha, beta, maximizingPlayer):
    valid_locations = get_valid_locations(grid)
    is_terminal = winning_move(grid, PLAYER_HUMAN) or winning_move(grid, PLAYER_AI) or is_board_full(grid)
    
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(grid, PLAYER_AI):
                return (None, 100000000000)
            elif winning_move(grid, PLAYER_HUMAN):
                return (None, -100000000000)
            else: # Game is over, no more valid moves
                return (None, 0)
        else: # Depth is zero
            return (None, score_position(grid, PLAYER_AI))

    if maximizingPlayer:
        value = -math.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(grid, col)
            temp_grid = grid.copy()
            drop_disc(temp_grid, row, col, PLAYER_AI)
            new_score = minimax(temp_grid, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                best_col = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return best_col, value

    else: # Minimizing player
        value = math.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(grid, col)
            temp_grid = grid.copy()
            drop_disc(temp_grid, row, col, PLAYER_HUMAN)
            new_score = minimax(temp_grid, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                best_col = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return best_col, value


### Implementation of the alpha-beta pruning  

The Alpha-Beta pruning is already integrated into the Minimax function above by maintaining alpha (the best score for the maximizing player) and beta (the best score for the minimizing player) and pruning branches that cannot improve the outcome.

### Choice and implementation of the Static Evaluation Function.

The static evaluation function score_position assesses the game state by analyzing windows of four consecutive slots on the grid. It assigns scores based on potential win, block, or strategic placements, especially favoring the central column, which gives more potential connections.

In [4]:
#Code block - Start the game
import random

def play_game():
    grid = create_grid()
    game_over = False
    turn = random.choice([PLAYER_HUMAN, PLAYER_AI])  # Randomly choose who starts

    print("Game Start! Human is Player 1, AI is Player 2.")
    
    while not game_over:
        print_grid(grid)
        if turn == PLAYER_HUMAN:
            # Human player turn
            col = int(input("Player 1 Make your Selection (0-6): "))
            
            if is_valid_location(grid, col):
                row = get_next_open_row(grid, col)
                drop_disc(grid, row, col, PLAYER_HUMAN)

                if winning_move(grid, PLAYER_HUMAN):
                    print("Player 1 wins!")
                    game_over = True
                elif is_board_full(grid):
                    print("It's a draw!")
                    game_over = True

                turn = PLAYER_AI  # Switch to AI turn
            else:
                print("Invalid selection. Try again.")

        else:
            # AI player turn
            print("AI is making a move...")
            col, minimax_score = minimax(grid, 3, -math.inf, math.inf, True)
            
            if is_valid_location(grid, col):
                row = get_next_open_row(grid, col)
                drop_disc(grid, row, col, PLAYER_AI)

                if winning_move(grid, PLAYER_AI):
                    print("AI wins!")
                    game_over = True
                elif is_board_full(grid):
                    print("It's a draw!")
                    game_over = True

                turn = PLAYER_HUMAN  # Switch back to human player

    print_grid(grid)
    print("Game over!")

# Call the function to start the game
play_game()


Game Start! Human is Player 1, AI is Player 2.
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]]
AI is making a move...
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]]
AI is making a move...
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 1. 2. 0. 0. 0.]]
AI is making a move...
[[0. 0. 0. 0. 